In [1]:
import numpy as np

In [29]:
def PrettyPrintBinary(state):
    outstr = ""
    if (type(state) == list and type(state[0]) == tuple):
        for i in range(len(state)):
            outstr += (str)(state[i][0])
            outstr += " |"
            outstr += state[i][1]
            outstr += "> "
            if (i < len(state) - 1):
                outstr += "+ "
        print(outstr)
    # elif (type(state) == list and type(state[0]) == np.float64):
    #     sto = 0
    #     for s in range(len(state)):
    #         if (len(bin(s)) > sto):
    #             sto = len(bin(s))
    #     print(sto)
    #     for s in range(len(state)):
    #         if(state[s] != 0):
    #             outstr += str(state[s])
    #             outstr += "|"
    #             addstr = str(bin(s))[2:]
    #             if (len(addstr) < sto - 2):
                    
    #             outstr += str(bin(s))[2:]
    #             outstr += "> "
    #     print(outstr)
    else:
        print("State is not valid")
def PrettyPrintInteger(state):
    if (type(state) == list and type(state[0]) == np.float64):
        outstr = ""
        for i in range(len(state)):
            if (state[i] != 0):
                if (i != 0):
                    outstr += "+ "
                outstr += (str)(state[i])
                outstr += " |"
                outstr += str(i)
                outstr += "> "
                # if (i < len(state) - 1):
                #     outstr += "+ "
        print(outstr)
    else:
        print("State passed to PrettyPrintInteger is not valid")

def VecToDirac(myVec):
    if ((type(myVec) == list and type(myVec[0]) == np.float64 and len(myVec) % 2 == 0)):
        newState = []
        binlen = int(np.log2(len(myVec)))
        for i in range(len(myVec)):
            if (myVec[i] != 0):
                bi = bin(i)[2:].zfill(binlen)
                newState.append((myVec[i], bi))
        return newState
    else:
        print("Error in VecToDirac")
        
def DiracToVec(state):
    if (not (type(state) == list and type(state[0]) == tuple)):
        print("Error in DiracToVec")
    else:
        newVec = []
        for i in range(2**len(state[0][1])):
            newVec.append(0)
        for s in state:
            newVec[int(s[1], 2)] = s[0]
        return newVec


In [30]:
myState2=[
   (np.sqrt(0.1)*1.j, '101'),
   (np.sqrt(0.5), '000') ,
   (-np.sqrt(0.4), '010' )]
myEquivilentState2 = [np.sqrt(0.5), 0, -np.sqrt(0.4), 0, 0, np.sqrt(0.1)*1.j, 0, 0]
PrettyPrintBinary(myState2)
PrettyPrintInteger(myEquivilentState2)
PrettyPrintBinary(VecToDirac(myEquivilentState2))
PrettyPrintInteger(DiracToVec(myState2))
PrettyPrintBinary(VecToDirac(DiracToVec(myState2)))

0.31622776601683794j |101> + 0.7071067811865476 |000> + -0.6324555320336759 |010> 
0.7071067811865476 |0> + -0.6324555320336759 |2> + 0.31622776601683794j |5> 
0.7071067811865476 |000> + -0.6324555320336759 |010> + 0.31622776601683794j |101> 
0.7071067811865476 |0> + -0.6324555320336759 |2> + 0.31622776601683794j |5> 
0.7071067811865476 |000> + -0.6324555320336759 |010> + 0.31622776601683794j |101> 


In [58]:
# Simulator S
class S:
    def H(wire, inputState):
        newState = []
        for s in inputState:
            if (s[1][wire] == '0'):
                bi = list(s[1])
                bi[wire] = '0'
                newState.append((s[0] * np.sqrt(1/2), "".join(bi)))
                bi[wire] = '1'
                newState.append((s[0] * np.sqrt(1/2), "".join(bi)))
            if (s[1][wire] == '1'):
                bi = list(s[1])
                bi[wire] = '0'
                newState.append((s[0] * np.sqrt(1/2), "".join(bi)))
                bi[wire] = '1'
                newState.append((s[0] * -np.sqrt(1/2), "".join(bi)))
        return newState

    def Phase(wire, theta, inputState):
        newState = []
        for s in inputState:
            if (s[1][wire] == '0'):
                newState.append(s)
            else:
                newState.append((s[0] * np.exp(1.j * theta), s[1]))
        return newState

    def CNOT(controlwire, notwire, inputState):
        newState = []
        for s in inputState:
            if (s[1][controlwire] == '0'):
                newState.append(s)
            else:
                bi = list(s[1])
                bi[notwire] = str((int(bi[notwire]) + 1) % 2)
                # if (bi[notwire] == '0'):
                #     print("here")
                #     bi[notwire] == '1'
                # else:
                #     bi[notwire] == '0'
                string = "".join(bi)                
                newState.append((s[0], string))
        return newState

In [59]:
PrettyPrintBinary(S.H(0,myState2))
PrettyPrintBinary(myState2)
PrettyPrintBinary(S.Phase(1, np.pi, myState2))
PrettyPrintBinary(S.CNOT(0, 1, myState2))

0.223606797749979j |001> + (-0-0.223606797749979j) |101> + 0.5000000000000001 |000> + 0.5000000000000001 |100> + -0.447213595499958 |010> + -0.447213595499958 |110> 
0.31622776601683794j |101> + 0.7071067811865476 |000> + -0.6324555320336759 |010> 
0.31622776601683794j |101> + 0.7071067811865476 |000> + (0.6324555320336759-7.745346429080775e-17j) |010> 
0.31622776601683794j |111> + 0.7071067811865476 |000> + -0.6324555320336759 |010> 


A NOT gate is H P(pi) H
